<font size="30">Análise Comparativa dos Dados</font>

## 1. Obtenção de Dados

### 1.1 Importação de bibliotecas

In [276]:
import pandas as pd
from IPython.display import display, Markdown, HTML
import joblib
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
import warnings
from sklearn.metrics import classification_report
from sklearn.exceptions import UndefinedMetricWarning

from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import ShuffleSplit, GridSearchCV, KFold, cross_validate
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

### 1.2 Importação dos Dados

Nessa etapa obteremos novamente os arquivos brutos de dados e o dicionário antes de iniciar o pre-processamento, pois serão utilizados além de serem necessários para visualização

In [255]:
caminho = '../data/raw/Orange_Quality_Data.csv'
laranjas = pd.read_csv(caminho) #Obtendo o dataset

laranjas

,Size (cm),Weight (g),Brix (Sweetness),pH (Acidity),Softness (1-5),HarvestTime (days),Ripeness (1-5),Color,Variety,Blemishes (Y/N),Quality (1-5)
0,7.5,180,12.0,3.2,2.0,10,4.0,Orange,Valencia,N,4.0
1,8.2,220,10.5,3.4,3.0,14,4.5,Deep Orange,Navel,N,4.5
2,6.8,150,14.0,3.0,1.0,7,5.0,Light Orange,Cara Cara,N,5.0
3,9.0,250,8.5,3.8,4.0,21,3.5,Orange-Red,Blood Orange,N,3.5
4,8.5,210,11.5,3.3,2.5,12,5.0,Orange,Hamlin,Y (Minor),4.5
...,...,...,...,...,...,...,...,...,...,...,...
236,8.0,194,10.9,3.6,5.0,13,1.0,Orange-Red,Tangerine,Y (Scars),5.0
237,7.4,275,8.5,3.5,5.0,20,5.0,Light Orange,Minneola (Hybrid),N,4.0
238,7.5,196,15.7,3.0,3.0,13,3.0,Deep Orange,Temple,Y (Minor Insect Damage),5.0
239,7.2,251,9.8,4.3,3.0,23,1.0,Light Orange,Moro (Blood),Y (Minor Insect Damage),3.0


In [259]:
dicionario = pd.read_csv("../data/external/dicionario.csv")
dicionario

,variavel,descrição,tipo,subtipo
0,Size (cm),Tamanho da fruta em cm,Quantitativa,Contínua
1,Weight (g),Peso da fruta em g,Quantitativa,Contínua
2,Brix (Sweetness),Nível de doçura,Quantitativa,Contínua
3,pH (Acidity),Nível de acidez em pH,Quantitativa,Contínua
4,Softness (1-5),Maciez de 1-5,Qualitativa,Ordinal
5,HarvestTime (days),Dias desde a colheita,Quantitativa,Discreta
6,Ripeness (1-5),Maduração de 1-5,Qualitativa,Ordinal
7,Color,Cor da laranja,Qualitativa,Nominal
8,Variety,Variedade da laranja,Qualitativa,Nominal
9,Blemishes (Y/N),Defeito,Qualitativa,Nominal


In [260]:
laranjas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Size (cm)           241 non-null    float64
 1   Weight (g)          241 non-null    int64  
 2   Brix (Sweetness)    241 non-null    float64
 3   pH (Acidity)        241 non-null    float64
 4   Softness (1-5)      241 non-null    float64
 5   HarvestTime (days)  241 non-null    int64  
 6   Ripeness (1-5)      241 non-null    float64
 7   Color               241 non-null    object 
 8   Variety             241 non-null    object 
 9   Blemishes (Y/N)     241 non-null    object 
 10  Quality (1-5)       241 non-null    float64
dtypes: float64(6), int64(2), object(3)
memory usage: 20.8+ KB


- Observamos, então, que o conjunto de dados está completo e não apresenta valores ausentes.

---

## 2. Preparação de Dados

### 2.1 Tratamento de colunas

Aqui realizamos a normalização, codificação e o tratamento de dados discrepantes e/ou faltantes dentro do conjunto de dados. 

A coluna target é uma variável qualitativa ordinal, mas expressa por números float de 0 a 5, que indicam uma classificação. Para que o modelo consiga trabalhar sem problemas, optamos por substituir os valores numéricos por textos em todas as variáveis qualitativas ordinais que são float.  

- Primeiramente verificamos os valores que existem nas colunas:

In [263]:
quality_unique = laranjas["Quality (1-5)"].unique()
quality_unique

array([4. , 4.5, 5. , 3.5, 1. , 3. , 2.5, 2. ])

In [267]:
softness_unique = laranjas["Softness (1-5)"].unique()
softness_unique

array([2. , 3. , 1. , 4. , 2.5, 3.5, 1.5, 5. , 4.5])

In [268]:
ripeness_unique = laranjas["Ripeness (1-5)"].unique()
ripeness_unique

array([4. , 4.5, 5. , 3.5, 2. , 3. , 2.5, 1. ])

- Aqui mudamos os valores para <object>:

In [269]:
# Função para mapear os valores de Quality para as palavras correspondentes
def classify_quality(value):
    if 0 <= value <= 1:
        return 'péssimo'
    elif 1.1 <= value <= 2:
        return 'ruim'
    elif 2.1 <= value <= 3:
        return 'regular'
    elif 3.1 <= value <= 4:
        return 'bom'
    elif 4.1 <= value <= 5:
        return 'ótimo'
    else:
        return 'valor inválido'  # Caso queira tratar valores fora do intervalo

# Aplicando a função à coluna Quality
laranjas['Quality (1-5)'] = laranjas['Quality (1-5)'].apply(classify_quality)

---

In [270]:
def classify_softness(value):
    if 0 <= value <= 1:
        return 'Muito Dura'
    elif 1.1 <= value <= 2:
        return 'Dura'
    elif 2.1 <= value <= 3:
        return 'Maciez Média'
    elif 3.1 <= value <= 4:
        return 'Macia'
    elif 4.1 <= value <= 5:
        return 'Muito Macia'
    else:
        return 'Valor Inválido'  # Caso queira tratar valores fora do intervalo

# Aplicando a função à coluna Quality para criar a coluna Maciez
laranjas['Softness (1-5)'] = laranjas['Softness (1-5)'].apply(classify_softness)

---

In [241]:
def classify_maturation(value):
    if 0 <= value <= 1:
        return 'Verde'
    elif 1.1 <= value <= 2:
        return 'Quase Maduro'
    elif 2.1 <= value <= 3:
        return 'Maduro'
    elif 3.1 <= value <= 4:
        return 'Bem maduro'
    elif 4.1 <= value <= 5:
        return 'Muito Maduro'
    else:
        return 'Valor Inválido'  # Caso queira tratar valores fora do intervalo

# Aplicando a função à coluna Quality para criar a coluna Maturação
laranjas['Ripeness (1-5)'] = laranjas['Ripeness (1-5)'].apply(classify_maturation)

In [219]:
laranjas.head(10)

,Size (cm),Weight (g),Brix (Sweetness),pH (Acidity),Softness (1-5),HarvestTime (days),Ripeness (1-5),Color,Variety,Blemishes (Y/N),Quality (1-5)
0,7.5,180,12.0,3.2,Dura,10,Bem maduro,Orange,Valencia,N,bom
1,8.2,220,10.5,3.4,Maciez Média,14,Muito Maduro,Deep Orange,Navel,N,ótimo
2,6.8,150,14.0,3.0,Muito Dura,7,Muito Maduro,Light Orange,Cara Cara,N,ótimo
3,9.0,250,8.5,3.8,Macia,21,Bem maduro,Orange-Red,Blood Orange,N,bom
4,8.5,210,11.5,3.3,Maciez Média,12,Muito Maduro,Orange,Hamlin,Y (Minor),ótimo
5,6.7,126,9.1,3.0,Dura,25,Quase Maduro,Orange,Navel,N,péssimo
6,7.2,160,9.0,3.5,Macia,9,Bem maduro,Yellow-Orange,Tangelo (Hybrid),N,bom
7,6.5,130,13.5,2.8,Dura,5,Muito Maduro,Light Orange,Murcott (Hybrid),N,ótimo
8,8.8,240,7.5,4.0,Muito Macia,18,Maduro,Deep Orange,Moro (Blood),Y (Sunburn),regular
9,7.8,190,12.0,3.1,Dura,11,Muito Maduro,Orange,Jaffa,N,ótimo


### 2.2 Pré-processamento

- Esse bloco de código está dividindo o dataset em duas partes: as características (features) e a variável alvo (target). 
- Essa separação é um passo comum no pré-processamento de dados para problemas de aprendizado supervisionado, onde X são os dados de entrada e y é o rótulo ou valor que queremos prever.

In [271]:
X = laranjas.drop('Quality (1-5)', axis=1)
# 'X' contém todas as colunas do dataset, exceto a coluna alvo 'Quality (1-5)'

y = laranjas['Quality (1-5)']
# 'y' contém apenas a coluna alvo 'Quality (1-5)', que é a variável que queremos prever

---

Dividindo o dataset em conjuntos de treino e teste
- 'X_train' e 'y_train' são os dados de treino (80% dos dados)
- 'X_test' e 'y_test' são os dados de teste (20% dos dados)
- O parâmetro 'test_size=0.2' indica que 20% dos dados serão usados para teste
- 'random_state=42' garante que a divisão dos dados seja reprodutível e consistente em execuções diferentes

Dividir os dados em conjuntos de treino e teste é um passo fundamental na construção de modelos de machine learning. Isso permite avaliar o desempenho do modelo em dados não vistos, ajudando a detectar problemas de overfitting (quando o modelo se ajusta demais aos dados de treino e não generaliza bem para novos dados).

Usar uma divisão como 80/20 ou 70/30 é uma prática comum para garantir que o modelo tenha dados suficientes para aprender e ainda possa ser validado adequadamente.

In [243]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

---

Esse bloco de código define os passos de pré-processamento para diferentes tipos de variáveis e cria um pipeline para aplicar essas transformações, seguido pelo treinamento de um modelo.

- Esse pipeline é uma abordagem comum para estruturar e organizar o fluxo de trabalho de machine learning. Ele automatiza o processo de preparação de dados e treinamento de modelo, garantindo que as mesmas transformações sejam aplicadas consistentemente nos dados de treino e teste.

- Facilita a manutenção do código e a experimentação, permitindo facilmente substituir o modelo de classificação ou ajustar as etapas de pré-processamento.

In [273]:
# Lista colunas categóricas e numéricas
# Separa as características em diferentes categorias para aplicar transformações específicas
num_features = ['Size (cm)', 'Weight (g)', 'Brix (Sweetness)', 'pH (Acidity)', 'HarvestTime (days)']
ordinal_features = ['Softness (1-5)', 'Ripeness (1-5)']
categorical_features = ['Color', 'Variety', 'Blemishes (Y/N)']

# Preprocessador para dados numéricos
# Pipeline de transformação para colunas numéricas:
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Atualize o `OrdinalEncoder` para lidar com novas categorias
# Pipeline de transformação para colunas ordinais:
ordinal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

# Preprocessador para colunas categóricas nominais
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Atualiza o preprocessor com o novo ordinal_transformer
# ColumnTransformer aplica diferentes transformações às colunas especificadas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_features),
        ('ord', ordinal_transformer, ordinal_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Cria o pipeline final com o modelo
# Pipeline que primeiro aplica o pré-processamento e depois treina o modelo de classificação
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])

---

- Ao encapsular tanto o pré-processamento quanto o modelo de classificação dentro de um Pipeline, você assegura que todas as etapas necessárias sejam executadas de maneira coesa e ordenada.
- Isso é especialmente útil ao avaliar o modelo com validação cruzada, pois garante que as etapas de pré-processamento sejam aplicadas aos dados de treino em cada iteração, evitando vazamento de dados (data leakage) e proporcionando uma avaliação mais honesta da performance do modelo.
- Facilita a manutenção e a modificação do pipeline, permitindo que você altere ou ajuste as etapas individuais de pré-processamento ou escolha diferentes modelos de classificação sem alterar significativamente a estrutura do código.

In [274]:
# Criar o pipeline completo
# Definição do pipeline principal, que engloba as etapas de pré-processamento e o modelo de classificação final
# O pipeline combina os seguintes passos:
# - 'preprocessor': Responsável pelo pré-processamento dos dados, incluindo tratamento de valores ausentes, codificação de variáveis categóricas, 
# escalonamento de variáveis numéricas e codificação de variáveis ordinais. As transformações específicas para cada tipo de dado foram definidas anteriormente no `preprocessor`.

# - 'classifier': Modelo de classificação que será treinado após o pré-processamento dos dados. Neste caso, um `RandomForestClassifier` é utilizado, 
# que é um modelo de árvore de decisão em conjunto (ensemble) que pode lidar bem com dados complexos e evitar overfitting.

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])

---

- Objetivo: Treinar o pipeline (pré-processamento + modelo) com os dados de treinamento.
- Função: Aplica pré-processamento aos dados de entrada e ajusta o modelo de classificação para prever a variável alvo.
- Benefício: Automatiza o fluxo de trabalho de treinamento, assegurando consistência e evitando vazamento de dados.

In [275]:
# O método .fit() é usado para treinar o pipeline inteiro nos dados de treinamento fornecidos.
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Size (cm)', 'Weight (g)',
                                                   'Brix (Sweetness)',
                                                   'pH (Acidity)',
                                                   'HarvestTime (days)']),
                                                 ('ord',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ordinal',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=-1))]),
                                                  ['Softness (1-5)',
                                                   'Ripeness (1-5)']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Color', 'Variety',
                                                   'Blemishes (Y/N)'])])),
                ('classifier', RandomForestClassifier())])

In [277]:
# Configurar para ignorar UndefinedMetricWarning
warnings.simplefilter(action='ignore', category=UndefinedMetricWarning)

# treinar e avaliar o modelo
pipeline.fit(X_train, y_train)

# Avaliar modelo com dados de teste
y_pred = pipeline.predict(X_test)

# Imprimir o relatório de classificação, tratando de evitar problemas com classes não previstas
print(classification_report(y_test, y_pred, zero_division=0))

              precision    recall  f1-score   support

         bom       0.67      0.85      0.75        34
     péssimo       0.00      0.00      0.00         1
     regular       0.57      0.44      0.50         9
        ruim       0.00      0.00      0.00         4
       ótimo       0.80      0.64      0.71        25

    accuracy                           0.67        73
   macro avg       0.41      0.39      0.39        73
weighted avg       0.66      0.67      0.66        73



In [278]:
# Configurações do experimento
n_splits_comparative_analysis = 10
n_folds_grid_search = 5
test_size = 0.3
random_state = 42
scoring = 'accuracy'

# Métricas para análises
metrics = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']

# Configurações do modelo
models = [
    ('Gaussian Naive Bayes', GaussianNB(), {'var_smoothing': [10**(-9)]}),
    ('Decision Tree', DecisionTreeClassifier(random_state=random_state), {'criterion': ['gini', 'entropy'], 'max_depth': [3, 6, 8]}),
    ('Random Forest', RandomForestClassifier(random_state=random_state), {'criterion': ['gini', 'entropy'], 'max_depth': [3, 6, 8], 'n_estimators': [10, 30]}),
    ('Logistic Regression', LogisticRegression(max_iter=1000), {'C': np.logspace(-4, 4, 20)}),
]

# Definir pré-processamento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), ['Size (cm)', 'Weight (g)', 'Brix (Sweetness)', 'pH (Acidity)', 'HarvestTime (days)']),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))  # Definindo sparse=False para garantir matriz densa
        ]), ['Softness (1-5)', 'Ripeness (1-5)', 'Color', 'Variety', 'Blemishes (Y/N)'])
    ])

# Dividir dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

results = pd.DataFrame({})
cross_validate_grid_search = KFold(n_splits=n_folds_grid_search)
cross_validate_comparative_analysis = ShuffleSplit(n_splits=n_splits_comparative_analysis, test_size=test_size, random_state=random_state)

for model_name, model_object, model_parameters in models:
    print(f"Running {model_name}...")
    
    model_grid_search = GridSearchCV(
        estimator=model_object,
        param_grid=model_parameters,
        scoring=scoring,
        n_jobs=-1,
        cv=cross_validate_grid_search
    )
    
    approach = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model_grid_search)
    ])
    
    try:
        scores = cross_validate(
            estimator=approach,
            X=X_train,
            y=y_train,
            cv=cross_validate_comparative_analysis,
            n_jobs=-1,
            scoring=metrics
        )
        
        scores['model_name'] = [model_name] * n_splits_comparative_analysis
        df_scores = pd.DataFrame(scores)
        df_scores = df_scores.drop(columns=['model_name'])
        df_scores = df_scores.agg(['mean', 'std'])
        
        print(f"Results for {model_name}:")
        display(df_scores)
        
        results = pd.concat([results, pd.DataFrame(scores)], ignore_index=True)
    except Exception as e:
        print(f"Error running {model_name}: {e}")

# Mostrar resultados finais
print("Final results:")
display(results)

# Avaliar modelo com dados de teste
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestClassifier(random_state=random_state))  # Substitua pelo melhor modelo encontrado
])
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

Running Gaussian Naive Bayes...


/mnt/c/Users/bruno/AvantiBootcamp/equipe7-2024.2/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/mnt/c/Users/bruno/AvantiBootcamp/equipe7-2024.2/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/mnt/c/Users/bruno/AvantiBootcamp/equipe7-2024.2/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

Results for Gaussian Naive Bayes:


,fit_time,score_time,test_accuracy,test_precision_macro,test_recall_macro,test_f1_macro
mean,0.073824,0.019164,0.468627,0.376676,0.394893,0.365427
std,0.021499,0.009451,0.064338,0.102237,0.115678,0.096277


Running Decision Tree...
Results for Decision Tree:


...

/mnt/c/Users/bruno/AvantiBootcamp/equipe7-2024.2/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/mnt/c/Users/bruno/AvantiBootcamp/equipe7-2024.2/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,fit_time,score_time,test_accuracy,test_precision_macro,test_recall_macro,test_f1_macro
mean,2.568276,0.017637,0.601961,0.408133,0.407692,0.394106
std,0.313789,0.006223,0.050669,0.084229,0.060919,0.057929


Final results:


,fit_time,score_time,test_accuracy,test_precision_macro,test_recall_macro,test_f1_macro,model_name
0,0.057501,0.016643,0.411765,0.271634,0.293567,0.276608,Gaussian Naive Bayes
1,0.059538,0.008806,0.509804,0.304545,0.301993,0.295556,Gaussian Naive Bayes
2,0.089251,0.017950,0.470588,0.396667,0.392381,0.378788,Gaussian Naive Bayes
3,0.068841,0.016947,0.470588,0.461477,0.427619,0.428739,Gaussian Naive Bayes
4,0.100259,0.034808,0.509804,0.356643,0.466667,0.384000,Gaussian Naive Bayes
5,0.079948,0.022215,0.568627,0.476341,0.550700,0.478770,Gaussian Naive Bayes
6,0.051413,0.014412,0.352941,0.253070,0.296667,0.250826,Gaussian Naive Bayes
7,0.039615,0.007823,0.450980,0.458129,0.327611,0.357676,Gaussian Naive Bayes
8,0.091494,0.035625,0.411765,0.257400,0.285147,0.264806,Gaussian Naive Bayes
9,0.100381,0.016414,0.529412,0.530849,0.606579,0.538504,Gaussian Naive Bayes


              precision    recall  f1-score   support

         bom       0.67      0.85      0.75        34
     péssimo       0.00      0.00      0.00         1
     regular       0.57      0.44      0.50         9
        ruim       0.00      0.00      0.00         4
       ótimo       0.76      0.64      0.70        25

    accuracy                           0.67        73
   macro avg       0.40      0.39      0.39        73
weighted avg       0.65      0.67      0.65        73



In [230]:
def highlight_best(s, props=''):
    if s.name[1] != 'std':
        if s.name[0].endswith('time'):
            return np.where(s == np.nanmin(s.values), props, '')
        return np.where(s == np.nanmax(s.values), props, '')

display(Markdown("### 3.3 Resultados gerais e discussão"))
(
    results
    .groupby('model_name')
    .agg(['mean', 'std']).T
    .style
    .apply(highlight_best, props='color:white;background-color:gray;font-weight: bold;', axis=1)
    .set_table_styles([{'selector': 'td', 'props': 'text-align: center;'}])
)

### 3.3 Resultados gerais e discussão

## Configuração do experimento

## Resultados 

## Discussão